# Preprocessing

## Environment & imports

In [1]:
# ---------------------------------------------
# 0)  Install (first‐time only) & import libs
# ---------------------------------------------
# !pip install -q datasets transformers emoji==2.10.0 tqdm

from pathlib import Path
import re
import random
import json
from collections import defaultdict
from typing import List, Dict, Tuple

import emoji
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch

/home/drl-68/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load SAMSum

In [2]:
# ---------------------------------------------------------
# 1) Load SAMSum — 14 732 / 819 / 818 dialogues
# ---------------------------------------------------------
raw_ds: DatasetDict = load_dataset("samsum")
print({k: len(v) for k, v in raw_ds.items()})

Using the latest cached version of the module from /home/drl-68/.cache/huggingface/modules/datasets_modules/datasets/samsum/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e (last modified on Mon May  5 10:16:55 2025) since it couldn't be found locally at samsum, or remotely on the Hugging Face Hub.


{'train': 14732, 'test': 819, 'validation': 818}


## Build an emoji vocabulary and speaker token & Build / extend the tokenizer

count [UNK] occurrences in one HF Dataset

In [3]:
from tqdm import tqdm
import numpy as np
import torch

def count_unk(ds, tokenizer, field="dialogue", batch_size=1024):
    unk_id = tokenizer.unk_token_id
    total_unk, total_tokens = 0, 0

    for i in tqdm(range(0, len(ds), batch_size), desc="Tokenising"):
        batch_texts = ds[i : i + batch_size][field]
        enc = tokenizer(batch_texts, add_special_tokens=True, padding=False, truncation=False)
        for ids in enc["input_ids"]:
            arr = np.array(ids)
            total_unk += np.sum(arr == unk_id)
            total_tokens += len(arr)
    return total_unk, total_tokens

BEFORE adding emojis

In [4]:
tok_base = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama_v1.1")
unk_stats_before = {}
for split in ["train", "validation", "test"]:
    unk_stats_before[split] = count_unk(raw_ds[split], tok_base)
print("\n[UNK] counts BEFORE adding emoji tokens")
for split, (u, t) in unk_stats_before.items():
    print(f"{split:<10}: {u:8d}  ({u/t:.3%} of tokens)")

Tokenising: 100%|██████████| 1/1 [00:00<00:00, 41.07it/s]


[UNK] counts BEFORE adding emoji tokens
train     :        0  (0.000% of tokens)
validation:        0  (0.000% of tokens)
test      :        0  (0.000% of tokens)


สร้าง EMOJI_TOKENS

In [5]:
# ถ้า kernel เพิ่งรีสตาร์ต ตัวแปรจะหายหมด
# สร้างชุด emoji ใหม่จาก raw_ds
from typing import List
import emoji

def extract_emojis(text: str) -> List[str]:
    return [ch for ch in text if ch in emoji.EMOJI_DATA]

emoji_set = set()
for split in ["train", "validation", "test"]:
    for dlg in raw_ds[split]["dialogue"]:
        emoji_set.update(extract_emojis(dlg))

EMOJI_TOKENS = sorted(emoji_set)          # ≈ 300-320 รายการ
print(f"Unique emojis found: {len(EMOJI_TOKENS)}")

Unique emojis found: 305


In [6]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

Extend tokenizer with emojis + speaker tags

In [7]:
from transformers import AutoTokenizer

# ---------- 1) โหลด tokenizer ดั้งเดิม ----------
tok_base = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama_v1.1")
vocab_orig = len(tok_base)

# ---------- 2) เตรียมชุด token ใหม่ ----------
#   • EMOJI_TOKENS  : ทุกอิโมจิที่ “พบอย่างน้อย 1 ครั้ง” ใน SAMSum
#   • SPEAKER_TOKENS: [S1] – [S10]
SPEAKER_TOKENS = [f"[S{i}]" for i in range(1, 11)]
new_tokens = EMOJI_TOKENS + SPEAKER_TOKENS

# ---------- 3) สร้าง tokenizer สำเนาแล้วเพิ่ม token ----------
tok_ext = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama_v1.1")
added = tok_ext.add_tokens(new_tokens)
vocab_new = len(tok_ext)

# ---------- 4) แสดงผล ----------
print(f"Original vocab size : {vocab_orig}")
print(f"Added new tokens     : {added}  "
      f"(emoji = {len(EMOJI_TOKENS)}, speaker = {len(SPEAKER_TOKENS)})")
print(f"New vocab size       : {vocab_new}")

# (Optional) พิมพ์ตัวอย่างอิโมจิ 20 ตัวแรก
print("\nFirst 20 emoji tokens:", EMOJI_TOKENS[:20])

tok_ext.save_pretrained("tokenizer_samsum_su")   # โฟลเดอร์ใหม่

Original vocab size : 32000
Added new tokens     : 315  (emoji = 305, speaker = 10)
New vocab size       : 32310

First 20 emoji tokens: ['‼', '⏱', '☀', '☂', '☔', '☕', '☘', '☝', '☠', '☢', '☹', '☺', '♀', '♂', '♥', '♻', '⚪', '⚫', '⚰', '⚽']


('tokenizer_samsum_su/tokenizer_config.json',
 'tokenizer_samsum_su/special_tokens_map.json',
 'tokenizer_samsum_su/tokenizer.model',
 'tokenizer_samsum_su/added_tokens.json',
 'tokenizer_samsum_su/tokenizer.json')

AFTER adding emojis

In [8]:
unk_stats_after = {}
for split in ["train", "validation", "test"]:
    unk_stats_after[split] = count_unk(raw_ds[split], tok_ext)
print("\n[UNK] counts AFTER adding emoji tokens")
for split, (u, t) in unk_stats_after.items():
    print(f"{split:<10}: {u:8d}  ({u/t:.3%} of tokens)")

Tokenising: 100%|██████████| 1/1 [00:00<00:00, 38.35it/s]


[UNK] counts AFTER adding emoji tokens
train     :        0  (0.000% of tokens)
validation:        0  (0.000% of tokens)
test      :        0  (0.000% of tokens)


reduction check in UNKs

In [9]:
print("\nΔ [UNK] (before ➜ after):")
for split in ["train", "validation", "test"]:
    u0, _ = unk_stats_before[split]
    u1, _ = unk_stats_after[split]
    print(f"{split:<10}: {u0-u1:+d}  fewer UNKs  (↓{(u0-u1)/u0:.2%})")


Δ [UNK] (before ➜ after):
train     : +0  fewer UNKs  (↓nan%)
validation: +0  fewer UNKs  (↓nan%)
test      : +0  fewer UNKs  (↓nan%)


/tmp/ipykernel_619269/1937097288.py:5: RuntimeWarning: invalid value encountered in scalar divide
  print(f"{split:<10}: {u0-u1:+d}  fewer UNKs  (↓{(u0-u1)/u0:.2%})")


## Preprocess SAMSum Dateset

Speaker-name mapping → [S#]

In [10]:
# ---------------------------------------------------------
# 4) Helper to replace speaker names by [S#]
# ---------------------------------------------------------
SPEAKER_RE = re.compile(r"^([^:]+):\s*(.*)$")

def map_speakers(dialogue: str, max_speakers: int = 10
                 ) -> Tuple[str, Dict[str, str]]:
    """
    Returns dialogue with names replaced by [S#] and a mapping dict.
    """
    speaker_map, next_id = {}, 1
    new_lines = []
    for line in dialogue.split("\n"):
        m = SPEAKER_RE.match(line)
        if not m:                # safety – keep line as is
            new_lines.append(line)
            continue
        name, utt = m.groups()
        if name not in speaker_map:
            if next_id > max_speakers:      # truncate extra speakers
                name_token = "[SUNK]"
            else:
                name_token = f"[S{next_id}]"
                speaker_map[name] = name_token
                next_id += 1
        new_lines.append(f"{speaker_map.get(name, '[SUNK]')}: {utt}")
    return "\n".join(new_lines), speaker_map


Insert [SEP] after every utterance

In [11]:
def add_sep_every_utt(dialogue: str) -> str:
    lines = [l + " [SEP]" for l in dialogue.split("\n") if l.strip()]
    return " ".join(lines)

Switching-Utterance corruption
- Hyper-parameters: Pu = 1.0, Pn = 0/1

โดยที่

Pu (permute-utterance prob.) ความน่าจะเป็นที่ แต่ละ utterance จะถูกเลือก ใส่ลงในชุดที่นำไปสับตำแหน่ง

- pu = 1.0 แสดงว่าบังคับเลือกทุกบรรทัดแล้วค่อยสับคำแบบสุ่ม

Pn (name-mask prob.) ความน่าจะเป็นที่ token [S#] ด้านหน้าจะถูกเปลี่ยนเป็น [MASK]

- pn = 0.0 แสดงว่า ไม่ mask, โมเดลเห็น speaker tag

- pn = 1.0 แสดงว่า mask หมด, บังคับดู context

In [12]:
def make_switching_utterance(dialogue: str,
                             pu: float = 1.0,
                             pn: float = 0.0,
                             rng: random.Random = random
                            ) -> Tuple[str, List[int]]:
    """
    • dialogue  - speaker-tokenised, SEP-inserted string
    • pu        - prob. an utterance is selected for permutation
    • pn        - prob. we MASK the speaker token (⇒ [MASK])
    Returns:
        corrupted_dialogue, labels_per_utt  (1 = permuted (สลับบทพูด), 0 = original)
    """
    # 1) split back into utterances
    utts = [u.strip() for u in dialogue.split("[SEP]") if u.strip()]
    idxs = list(range(len(utts)))

    # 2) pick indices to permute
    perm_idx = [i for i in idxs if rng.random() < pu]
    shuffled = perm_idx.copy()
    rng.shuffle(shuffled)                 # in-place
    perm_map = dict(zip(perm_idx, shuffled))

    # 3) build new utterance list, labels
    new_utts, labels = [], []
    for i in idxs:
        src = perm_map.get(i, i)          # swapped or same
        u = utts[src]
        # optionally mask speaker token ([S#]: → [MASK]:)
        if rng.random() < pn:
            u = re.sub(r"^\[S\d+\]", "[MASK]", u)
        new_utts.append(u)
        labels.append(int(src != i))      # 1 if permuted
    corrupted = " [SEP] ".join(new_utts) + " [SEP]"
    return corrupted, labels


## Switching-Utterance (SU) pre-training dataset

In [13]:
# ---------------------------------------------------------
# 7) Create HF Datasets with tokenised inputs, attention,
#    SEP positions, and per-utterance labels
# ---------------------------------------------------------
MAX_LEN = 512                          # paper setting
Pu, Pn = 1.0, 0.0                      # best config in Table 2


def preprocess_example(example, split):

    if tok_ext.pad_token is None:
        tok_ext.add_special_tokens({'pad_token': '[PAD]'})
        
    # a) replace speakers & add SEP
    dlg, _ = map_speakers(example["dialogue"])
    dlg = add_sep_every_utt(dlg)

    # b) corruption
    corrupted, labels = make_switching_utterance(dlg, Pu, Pn)

    # c) tokenize (truncate if >512 tokens)
    enc = tok_ext(corrupted,
              truncation=True, max_length=MAX_LEN,
              padding="max_length")
    
    # d) find SEP token positions (needed for loss later)
    sep_id = tok_ext("[SEP]")["input_ids"][0]
    sep_positions = [i for i, id_ in enumerate(enc["input_ids"])
                     if id_ == sep_id][:len(labels)]  # clip if truncated

    enc["labels"] = labels[:len(sep_positions)]
    enc["sep_positions"] = sep_positions
    enc["dialogue_len"] = len(labels)
    return enc

su_ds = DatasetDict()
for split in ["train", "validation", "test"]:
    su_ds[split] = raw_ds[split].map(
        preprocess_example,
        fn_kwargs={"split": split},
        remove_columns=raw_ds[split].column_names,
        desc=f"Building SU {split}"
    )

su_ds.save_to_disk("data/samsum_switching_utterance")
print(su_ds)

Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 221435.89 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'sep_positions', 'dialogue_len'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'sep_positions', 'dialogue_len'],
        num_rows: 818
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'sep_positions', 'dialogue_len'],
        num_rows: 819
    })
})


ไฟล์ Arrow ถูกบันทึกไว้ที่ data/samsum_switching_utterance/ พร้อมฟิลด์ input_ids / attention_mask / labels / sep_positions / dialogue_len.

## Self-supervised Pre-training

ใช้ Dataset เฉพาะส่วนของ train ของ SAMSum มาทำการ pre_train แล้วใช้ validation ไว้ดู early-stopping / tuning ส่วน test ต้องไม่ถูกแตะ เพื่อไม่ให้โมเดล “เห็น” บทสนทนาที่จะใช้วัด ROUGE ภายหลัง

In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

Imports & helpers

In [10]:
import math
import torch
import random
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    AutoModelForCausalLM
)
from datasets import load_from_disk

# -------------------------------
# CONFIG
# -------------------------------
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4 # paper 128
MAX_LEN    = 512 # paper 512
LR         = 3e-5
WARMUP     = 500
MAX_STEPS  = 40000

Dataset & collate

In [11]:
# -------------------------------
# LOAD DATASET
# -------------------------------
dataset = load_from_disk("data/samsum_switching_utterance")

# -------------------------------
# COLLATE FUNCTION
# -------------------------------
# def collate_fn(batch):
#     keys = ["input_ids", "token_type_ids", "attention_mask"]
#     inputs = {k: torch.tensor([b[k] for b in batch]) for k in keys}
#     labels = [torch.tensor(b["labels"], dtype=torch.float) for b in batch]
#     sep_pos = [torch.tensor(b["sep_positions"]) for b in batch]
#     return inputs, labels, sep_pos

def collate_fn(batch):
    keys = batch[0].keys()
    inputs = {
        k: torch.stack([torch.tensor(b[k]) if not isinstance(b[k], torch.Tensor) else b[k] for b in batch])
        for k in keys if k not in ["labels", "sep_positions"]
    }
    labels = [torch.tensor(b["labels"], dtype=torch.float) for b in batch]
    sep_pos = [torch.tensor(b["sep_positions"]) for b in batch]
    return inputs, labels, sep_pos



Model

In [12]:
# -------------------------------
# MODEL
# -------------------------------
class SepClassifier(nn.Module):
    def __init__(self, model_name="TinyLlama/TinyLlama_v1.1", dropout=0.1):
        super().__init__()
        config = AutoConfig.from_pretrained(model_name)
        self.llama = AutoModelForCausalLM.from_pretrained(
            model_name,
            config=config,
            # torch_dtype=torch.float16  # ใช้ float16 เฉพาะ LLaMA model
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(config.hidden_size, 1)  

    def forward(self, input_ids, attention_mask, sep_positions, **kwargs):
        # เรียก model โดยให้คืนค่า hidden states ทั้งหมด
        outputs = self.llama(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # เอา hidden state ชั้นสุดท้ายมาใช้
        hidden_states = outputs.hidden_states[-1]

        # ดึง hidden vectors ที่ตำแหน่ง sep ของแต่ละ sample
        sep_vecs = []
        for i, pos_tensor in enumerate(sep_positions):
            if pos_tensor.numel() == 0:
                continue  # ข้าม sample ที่ไม่มี SEP
            pos_tensor = pos_tensor.to(hidden_states.device).long()
            if torch.any(pos_tensor >= hidden_states.shape[1]):
                raise ValueError(f"Invalid sep_position index: {pos_tensor.tolist()} exceeds hidden size {hidden_states.shape[1]}")
            sep_vecs.append(hidden_states[i].index_select(0, pos_tensor))

        if not sep_vecs:
            raise ValueError("All sep_positions are empty. Cannot compute logits.")

        sep_vecs = torch.cat(sep_vecs, dim=0)
        sep_vecs = torch.nan_to_num(sep_vecs, nan=0.0, posinf=1e4, neginf=-1e4)
        sep_vecs = torch.clamp(sep_vecs, -1e4, 1e4)  # limit range

        # Apply dropout และ classifier
        logits = self.classifier(self.dropout(sep_vecs)).squeeze(-1)  # (total_seps,)
        logits = torch.clamp(logits, -30, 30)  # clamp เพื่อป้องกัน NaN ใน BCEWithLogitsLoss
        return logits


In [13]:
import torch
import gc

# เคลียร์ cache ของ CUDA
torch.cuda.empty_cache()

# เคลียร์ object ที่ไม่ใช้งานแล้ว
gc.collect()

# (Optional) หากใช้ AMP หรือ context manager อื่นๆ
if torch.cuda.is_available():
    torch.cuda.ipc_collect()  # สำหรับเคลียร์ inter-process memory (เฉพาะบางเคส)


Training loop (train model until the train loss converged (upper bounded by 5k steps)

In [15]:
from tqdm import tqdm
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# -------------------------------
# INITIALIZATION
# -------------------------------
torch.cuda.empty_cache()
# Load tokenizer and ensure it has all required special tokens
tokenizer = AutoTokenizer.from_pretrained("tokenizer_samsum_su")

# Add special tokens if missing
special_tokens = {}
if tokenizer.pad_token is None:
    special_tokens["pad_token"] = "[PAD]"
if tokenizer.sep_token is None:
    special_tokens["sep_token"] = "[SEP]"

if special_tokens:
    tokenizer.add_special_tokens(special_tokens)

# Initialize model
model = SepClassifier()
for param in model.llama.parameters():  
    param.requires_grad = False

# Resize token embeddings to match new tokenizer length
model.llama.resize_token_embeddings(len(tokenizer))

# Setup DataLoader
# train_loader = DataLoader(
#     dataset["train"],
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     drop_last=True,
#     collate_fn=collate_fn
# )

train_loader = DataLoader(
    dataset["train"],
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn,
    num_workers=4,  # increase if you're using CPU-heavy preprocessing
    pin_memory=True  # if using GPU
)


# Optimizer, scheduler, loss
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
total_steps = min(10000, len(train_loader))
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=500,
    num_training_steps=total_steps
)
loss_fn = nn.BCEWithLogitsLoss()

# -------------------------------
# TRAINING LOOP
# -------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(next(model.parameters()).device)
step = 0
running_loss = 0.0
model.train()

from torch.amp import autocast, GradScaler

scaler = GradScaler(device='cuda')  # <-- add before your loop

for epoch in range(100):
    print(f"Epoch {epoch}")
    torch.cuda.empty_cache()

    with tqdm(train_loader, desc=f"Training Epoch {epoch}") as t_loader:
        for inputs, label_lists, sep_lists in t_loader:
            if step >= MAX_STEPS:
                break

            inputs = {k: v.to(device) for k, v in inputs.items()}
            sep_lists = [s.to(device) for s in sep_lists]
            flat_labels = torch.cat(label_lists).float().to(device)

            optimizer.zero_grad(set_to_none=True)

            with autocast(device_type='cuda'):
                logits = model(**inputs, sep_positions=sep_lists)
                loss = loss_fn(logits, flat_labels)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            running_loss += loss.item()
            step += 1

            t_loader.set_postfix(step=step, loss=loss.item(), avg_loss=running_loss / max(1, step % 100))
            if step % 100 == 0:
                print(f"[Step {step:4d}/{MAX_STEPS}] Avg Loss: {running_loss / 100:.4f}")
                running_loss = 0.0

    if step >= MAX_STEPS:
        break

# ------------------------
# SAVE MODEL
# ------------------------
torch.save(model.state_dict(), "llama_su_pretrained.pt")
print("Model saved to 'llama_su_pretrained.pt'")

cuda:0
Epoch 0


Training Epoch 0:   3%|▎         | 101/3683 [00:10<06:02,  9.88it/s, avg_loss=0.69, loss=0.69, step=101]

[Step  100/40000] Avg Loss: 0.6889


Training Epoch 0:   5%|▌         | 201/3683 [00:20<05:52,  9.88it/s, avg_loss=0.686, loss=0.686, step=201]

[Step  200/40000] Avg Loss: 0.6884


...

[Step 39800/40000] Avg Loss: 0.6407


Training Epoch 10:  83%|████████▎ | 3071/3683 [04:24<00:52, 11.63it/s, avg_loss=0.658, loss=0.619, step=39902]

[Step 39900/40000] Avg Loss: 0.6395


Training Epoch 10:  86%|████████▌ | 3170/3683 [04:32<00:44, 11.61it/s, avg_loss=63.8, loss=0.658, step=4e+4]  


[Step 40000/40000] Avg Loss: 0.6384
Model saved to 'llama_su_pretrained.pt'


In [16]:
torch.save(model.bert.state_dict(), "llama_su_pretrained.pt")

AttributeError: 'SepClassifier' object has no attribute 'bert'

Validation & early-stop (optional)

- Use the same DataLoader/loop on su_ds["validation"], compute average BCE loss; if it plateaus you can stop earlier than 5 k steps (what the authors mean by “until train loss converged”).

# Create Summarization Dataset

ขั้นตอนการทำ preprocess
1. โหลดชุดข้อมูล SAMSum
2. ทำ preprocessing:
    - แทนชื่อ speaker ด้วย [S1]–[S10]
    - เติม [SEP] ท้ายทุกประโยค
    - ใช้ tokenizer เดิมจาก pretraining (tokenizer_samsum_su)
    - truncate/pad ความยาวที่ max_length = 512
3. แปลงให้อยู่ในรูปแบบที่พร้อมใช้สำหรับ Seq2SeqTrainer
4. Save เป็นไฟล์ .pt หรือ DatasetDict ที่พร้อมใช้งาน

Load SAMSum Dataset

In [10]:
raw_ds: DatasetDict = load_dataset("samsum")
print({k: len(v) for k, v in raw_ds.items()})

{'train': 14732, 'test': 819, 'validation': 818}


Load Pretrained Tokenizer (same as used during pretraining)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer_samsum_su")
MAX_LEN = 512

Speaker Normalization Helpers

In [12]:
SPEAKER_RE = re.compile(r"^([^:]+):\s*(.*)$")

def map_speakers(dialogue: str, max_speakers: int = 10) -> Tuple[str, Dict[str, str]]:
    """
    Replace speaker names with generic [S1], [S2], ... tokens.
    """
    speaker_map, next_id = {}, 1
    new_lines = []
    for line in dialogue.split("\n"):
        m = SPEAKER_RE.match(line)
        if not m:
            new_lines.append(line)
            continue
        name, utt = m.groups()
        if name not in speaker_map:
            if next_id > max_speakers:
                name_token = "[SUNK]"
            else:
                name_token = f"[S{next_id}]"
                speaker_map[name] = name_token
                next_id += 1
        name_token = speaker_map.get(name, "[SUNK]")
        new_lines.append(f"{name_token}: {utt}")
    return "\n".join(new_lines), speaker_map

def add_sep_every_utt(dialogue: str) -> str:
    lines = [l + " [SEP]" for l in dialogue.split("\n") if l.strip()]
    return " ".join(lines)


Preprocessing Function

In [13]:
def preprocess_fn(example):
    normed_dialogue, _ = map_speakers(example["dialogue"])
    sep_dialogue = add_sep_every_utt(normed_dialogue)

    inputs = tokenizer(
        sep_dialogue,
        truncation=True,
        padding='max_length',
        max_length=MAX_LEN,
    )
    targets = tokenizer(
        example["summary"],
        truncation=True,
        padding='max_length',
        max_length=MAX_LEN,
    )

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"]
    }

Apply Preprocessing

In [14]:
tokenized_ds = raw_ds.map(preprocess_fn, batched=False)
tokenized_ds.save_to_disk("samsum_finetune_ready")
print("Preprocessed dataset saved to 'samsum_finetune_ready'")

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 167281.36 examples/s]

Preprocessed dataset saved to 'samsum_finetune_ready'


In [ ]:
# MAX_LEN = 512  # paper setting

# def preprocess_example(example, split):
#     # a) replace speakers & add SEP (same as pretraining)
#     dlg, _ = map_speakers(example["dialogue"])  # แปลงชื่อให้เป็น token สั้น ๆ เช่น <USR1>
#     dlg = add_sep_every_utt(dlg)                # เพิ่ม [SEP] ทุกท้ายประโยค

#     # b) tokenize dialogue input
#     enc = tok_base(dlg,
#               truncation=True,
#               max_length=MAX_LEN,
#               padding="max_length")

#     # c) tok_baseenize target summary
#     with tok_base.as_target_tokenizer():
#         summary = example["summary"]
#         summary_enc = tok_base(summary,
#                           truncation=True,
#                           max_length=MAX_LEN,
#                           padding="max_length")
    
#     # d) pack input and label
#     enc["labels"] = summary_enc["input_ids"]
#     return enc

# # สร้าง dataset ใหม่สำหรับ fine-tune
# finetune_ds = DatasetDict()
# for split in ["train", "validation", "test"]:
#     finetune_ds[split] = raw_ds[split].map(
#         preprocess_example,
#         fn_kwargs={"split": split},
#         remove_columns=raw_ds[split].column_names,
#         desc=f"Building Fine-tuning {split}"
#     )

# finetune_ds.save_to_disk("data/samsum_finetune")
# print(finetune_ds)

Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 126710.99 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
})


# Fine-tuning 


**เทียบกับ Paper**

| **Parameter**       | **Code**                          | **Paper (Section 3.2)**          | **เหมือน / ไม่เหมือน**        |
| ------------------- | --------------------------------- | -------------------------------- | --------------       |
| Model               | BERT2BERT (EncoderDecoderModel)   | BERT2BERT                        | เหมือน                 |
| Tokenizer           | bert-base-uncased + custom tokens | ใช้ tokenizer ดัดแปลง              | เหมือน              |
| Batch Size          | 8                                 | **16 (per step)**                | ไม่เหมือน → เล็กกว่า  |
| Epochs              | 3                                 | 3                                | เหมือน              |
| Learning Rate       | 5e-5                              | **3e-5**                         | ไม่เหมือน → สูงกว่า   |
| Warmup Steps        | 500                               | ใช้ scheduler (แต่ไม่ระบุ exact)     | เหมือน (สมเหตุสมผล) |
| Max Length (input)  | 512                               | 512                              | เหมือน              |
| Max Length (output) | 128                               | 128                              | เหมือน              |
| Beam Search         | 4                                 | 4                                | เหมือน              |

---


In [ ]:
import torch
from transformers import (
    BertTokenizerFast,
    EncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from datasets import load_from_disk

# ------------------------------
# Load processed dataset & tokenizer
# ------------------------------
dataset = load_from_disk("data/samsum_finetune_ready")
tokenizer = BertTokenizerFast.from_pretrained("tokenizer_samsum_su")

# ------------------------------
# Load pretrained EncoderDecoderModel
# ------------------------------
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "bert-base-uncased", "bert-base-uncased"
)
model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))

# Load your pretrained encoder weights
model.encoder.load_state_dict(torch.load("bert_su_pretrained.pt", map_location="cpu"))
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size
model.config.max_length = 128
model.config.num_beams = 4

# ------------------------------
# Define training arguments
# ------------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    evaluation_strategy="steps",
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    learning_rate=5e-5,
    warmup_steps=500,
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
)

# ------------------------------
# Data Collator & Trainer
# ------------------------------
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ------------------------------
# Start Training
# ------------------------------
trainer.train()
model.save_pretrained("bert_samsum_finetuned")
tokenizer.save_pretrained("tokenizer_samsum_su_finetune")

/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.cro

{'loss': 1.4418, 'grad_norm': 0.3646565079689026, 'learning_rate': 4.96e-05, 'epoch': 0.27}


                                                  
  9%|▉         | 501/5526 [01:41<2:22:17,  1.70s/it]

{'eval_loss': 0.22444657981395721, 'eval_runtime': 5.0061, 'eval_samples_per_second': 163.402, 'eval_steps_per_second': 20.575, 'epoch': 0.27}


 18%|█▊        | 1000/5526 [03:17<14:36,  5.17it/s] 

...

                                                   
100%|█████████▉| 5501/5526 [18:43<00:42,  1.69s/it]

{'eval_loss': 0.1668214201927185, 'eval_runtime': 4.9802, 'eval_samples_per_second': 164.251, 'eval_steps_per_second': 20.682, 'epoch': 2.99}


100%|██████████| 5526/5526 [18:50<00:00,  4.89it/s]


{'train_runtime': 1130.5819, 'train_samples_per_second': 39.091, 'train_steps_per_second': 4.888, 'train_loss': 0.28212739849194124, 'epoch': 3.0}


('tokenizer_samsum_su_finetune/tokenizer_config.json',
 'tokenizer_samsum_su_finetune/special_tokens_map.json',
 'tokenizer_samsum_su_finetune/vocab.txt',
 'tokenizer_samsum_su_finetune/added_tokens.json',
 'tokenizer_samsum_su_finetune/tokenizer.json')

## Evaluation

1. ROUGE (Recall-Oriented Understudy for Gisting Evaluation) ใช้วัดความคล้ายกันระหว่างสรุปที่โมเดลสร้างขึ้นกับสรุปอ้างอิง โดยเน้นไปที่ recall เป็นหลัก
	- ROUGE-1 (R-1) = Unigram overlap (คำเดี่ยว)
	- ROUGE-2 (R-2) = Bigram overlap (คำติดกัน 2 คำ)
	- ROUGE-L (R-L) = ใช้ Longest common subsequence (LCS) ในการวัดความคล้ายเชิงลำดับคำที่ยาวที่สุดที่ปรากฏในทั้งสองสรุป โดยคำนึงถึงลำดับคำด้วย

2. BLEU (Bilingual Evaluation Understudy) เดิมทีใช้ในงานแปลภาษา แต่ถูกประยุกต์ใช้ในงานสรุปข้อความได้เช่นกัน โดย BLEU จะเน้นการวัด precision คือดูว่า คำที่โมเดลสร้าง มีเท่าไรที่ตรงกับสรุปจริง ต่างจาก ROUGE ที่เน้น recall
	- BLEU วัดการทับซ้อนของ n-gram เช่น unigram, bigram, trigram
	- มีการใช้ brevity penalty หากสรุปสั้นกว่าที่ควรจะเป็น

3. BERTScore (BS) ใช้ embedding จากโมเดล BERT หรือ Transformer ตัวอื่น ๆ ในการวัด semantic similarity (ความใกล้เคียงด้านความหมาย) ระหว่างสรุปของโมเดลกับสรุปจริง โดยไม่จำเป็นต้องใช้คำเหมือนกันเป๊ะเหมือนกับ ROUGE หรือ BLEU แต่ BERTScore จะวัดว่าคำหรือวลีมีความหมายใกล้เคียงกันหรือไม่
	- วัดความคล้ายกันของคำใน embedding space เช่น "car" vs "vehicle" ก็ยังถือว่าใกล้เคียง
	- ใช้ precision / recall / F1 score ตามระยะห่างของ vector


In [1]:
from datasets import load_from_disk
from transformers import BertTokenizer, EncoderDecoderModel
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from tqdm import tqdm

/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# โหลด dataset
dataset = load_from_disk("data/samsum_finetune_ready")

# โหลดโมเดลและ tokenizer
tokenizer = BertTokenizer.from_pretrained('tokenizer_samsum_su_finetune')
model = EncoderDecoderModel.from_pretrained('bert_samsum_finetuned')

EncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [ ]:
# ย้ายโมเดลและข้อมูลไปยังอุปกรณ์ที่เหมาะสม (GPU หรือ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# วิเคราะห์ข้อมูลและทำการ summary ด้วย bert_samsum_finetuned
def generate_summary(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # ย้ายข้อมูลไปยัง device
    
    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'], 
            max_length=512, 
            num_beams=4, 
            early_stopping=True,
            decoder_start_token_id=model.config.decoder_start_token_id,  # กำหนดที่นี่
            pad_token_id=model.config.pad_token_id  # กำหนด pad_token_id ถ้าจำเป็น
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [4]:
# 1. ROUGE Score Calculation
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    
    return {key: sum(value)/len(value) for key, value in scores.items()}

# 2. BLEU Score Calculation
def calculate_bleu(predictions, references):
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = [ref.split()]
        bleu_scores.append(sentence_bleu(ref_tokens, pred_tokens))
    return sum(bleu_scores) / len(bleu_scores)

# 3. BERTScore Calculation
def calculate_bertscore(predictions, references):
    P, R, F1 = score(predictions, references, lang='en')
    return P.mean().item(), R.mean().item(), F1.mean().item()

# การทดสอบกับ dataset
def evaluate_model(dataset):
    predictions = []
    references = []
    
    # ใช้ข้อมูลจาก train สำหรับทำนาย และข้อมูลจาก test สำหรับการเปรียบเทียบ
    for i in tqdm(range(len(dataset['test'])), desc="Evaluating", unit="sample"):
        # สร้างสรุปจากโมเดล
        input_text = dataset['train'][i]['dialogue']  # ใช้ 'dialogue' จาก train เพื่อสร้างสรุป
        reference_summary = dataset['test'][i]['summary']  # ใช้ 'summary' จาก test เป็นสรุปจริง
        pred_summary = generate_summary(input_text)  # สร้างสรุปจากโมเดล
        
        predictions.append(pred_summary)
        references.append(reference_summary)
    
    # ROUGE Score
    rouge_scores = calculate_rouge(predictions, references)
    print("ROUGE Scores:", rouge_scores)

    # BLEU Score
    bleu_score = calculate_bleu(predictions, references)
    print("BLEU Score:", bleu_score)

    # BERTScore
    P, R, F1 = calculate_bertscore(predictions, references)
    print("BERTScore - Precision:", P, "Recall:", R, "F1:", F1)

# เรียกใช้งาน
evaluate_model(dataset)

Evaluating: 100%|██████████| 819/819 [39:14<00:00,  2.88s/sample]
/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluate

ROUGE Scores: {'rouge1': 0.08103071050965537, 'rouge2': 0.005501493938462314, 'rougeL': 0.07293283175759344}
BLEU Score: 8.859648156109322e-05


/home/drl-68/miniconda3/envs/samsum-bert/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.8399370312690735 Recall: 0.8468782305717468 F1: 0.843207597732544
